# TP de Machine Learning : Classification de Fruits avec K-Means

In [13]:
from PIL import Image
from os import listdir
from os.path import isfile, join
from random import choices
import matplotlib.pyplot as plt
from IPython.display import display

DATASET_DIR = "./fruits"

def take(nb_images: int): 
    files = [join(DATASET_DIR,f) for f in listdir(DATASET_DIR) if isfile(join(DATASET_DIR, f))]
    return [Image
            .open(f)
            .convert("RGB")
            .resize((100,100))
            for f in choices(files, k=nb_images)]


imgs = take(10)

TypeError: unsupported operand type(s) for +: 'int' and 'Image'